In [7]:
import os

import dotenv
import json

import pyeodh

dotenv.load_dotenv()


client = pyeodh.Client(
    username=os.getenv("ADES_USER"),
    token=os.getenv("ADES_TOKEN"),
)
ades = client.get_ades()

In [6]:
cwl_yaml = r"""cwlVersion: v1.0
$graph:
- class: CommandLineTool
  id: 's1_make_stac'
  inputs:
  - id: 'files'
    doc: FILES
    type:
      type: array
      items: File
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_make_stac@sha256:ae0c478b310ff24f8fe2427343364099594ae9566fffc22d5ded532544da776a
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
  outputs:
  - id: 'stac_catalog'
    outputBinding:
      glob: .
    type: Directory
- class: CommandLineTool
  id: 's1_process'
  inputs:
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'pair'
    inputBinding:
      prefix: --pair
    type: File
  - id: 'password'
    inputBinding:
      prefix: --password
    type:
    - 'null'
    - string
  - id: 'username'
    inputBinding:
      prefix: --username
    type:
    - 'null'
    - string
  outputs:
  - id: 'coherence'
    outputBinding:
      glob: data/results/coh_vv.tif
    type: File
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_process@sha256:db05e54041a45f453bed6f97d6c90653af7f9b94f566aaf7abc3d043ca3cbe43
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - /usr/local/bin/_entrypoint.sh
  - python
  - /app/app.py
- class: CommandLineTool
  id: 's1_search'
  inputs:
  - id: 'end_datetime'
    inputBinding:
      prefix: --end-datetime
    type:
    - 'null'
    - string
  - id: 'intersects'
    inputBinding:
      prefix: --intersects
    type:
    - 'null'
    - string
  - id: 'password'
    inputBinding:
      prefix: --password
    type:
    - 'null'
    - string
  - id: 'start_datetime'
    inputBinding:
      prefix: --start-datetime
    type:
    - 'null'
    - string
  - id: 'username'
    inputBinding:
      prefix: --username
    type:
    - 'null'
    - string
  outputs:
  - id: 'pairs'
    outputBinding:
      glob: pair_*.geojson
    type:
      items: File
      type: array
  requirements:
  - class: DockerRequirement
    dockerPull: ghcr.io/eo-datahub/user-workflows/s1_search@sha256:0428fb5081458b7b8d565796e7d73313bd8ebef0e6ba7a7962efcd08d2c01a8a
  - class: InlineJavascriptRequirement
  doc: "None\n"
  baseCommand:
  - python
  - /app/app.py
- class: Workflow
  id: 's1-coherence-d'
  inputs:
  - id: 'intersects'
    label: Intersects
    doc: "a GeoJSON-like json string, which provides a \"type\" member describing
      the type of the geometry and \"coordinates\"  member providing a list of coordinates.
      Will search for images intersecting this geometry.\n"
    default: "{\n  \"type\": \"Polygon\",\n  \"coordinates\": [\n    [\n      [0.08905898091569497,
      52.69722175598818],\n      [0.08905898091569497, 52.15527412683906],\n     \
      \ [0.9565339502005088, 52.15527412683906],\n      [0.9565339502005088, 52.69722175598818],\n\
      \      [0.08905898091569497, 52.69722175598818]\n    ]\n  ]\n}\n"
    type: string
  - id: 'start_datetime'
    label: Start datetime
    doc: Start datetime
    default: '2023-04-01'
    type: string
  - id: 'end_datetime'
    label: End datetime
    doc: End datetime
    default: '2023-06-30'
    type: string
  - id: 'username'
    label: Username
    doc: Username
    type: string
  - id: 'password'
    label: Password
    doc: Password
    type: string
  outputs:
  - id: 'stac_output'
    outputSource:
    - 's1_make_stac/stac_catalog'
    type: Directory
  requirements:
  - class: ResourceRequirement
    coresMin: 3
    ramMin: 15000
  - class: ScatterFeatureRequirement
  label: S1 coherence
  doc: Generate Sentinel 1 image pair coherence
  steps:
  - id: 's1_search'
    in:
    - id: 'intersects'
      source: 'intersects'
    - id: 'start_datetime'
      source: 'start_datetime'
    - id: 'end_datetime'
      source: 'end_datetime'
    - id: 'username'
      source: 'username'
    - id: 'password'
      source: 'password'
    out:
    - id: 'pairs'
    run: '#s1_search'
  - id: 's1_process'
    in:
    - id: 'pair'
      source: 's1_search/pairs'
    - id: 'intersects'
      source: 'intersects'
    - id: 'username'
      source: 'username'
    - id: 'password'
      source: 'password'
    out:
    - id: 'coherence'
    run: '#s1_process'
    scatter:
    - 'pair'
    scatterMethod: dotproduct
  - id: 's1_make_stac'
    in:
    - id: 'intersects'
      source: 'intersects'
    - id: 'files'
      source: 's1_process/coherence'
    out:
    - id: 'stac_catalog'
    run: '#s1_make_stac'
"""

In [ ]:
try:
    ades.get_process("s1-coherence-d").delete()
except Exception:
    print("Process not found, no need to undeploy.")

process = ades.deploy_process(cwl_yaml=cwl_yaml)

In [ ]:
intersects = {
    "type": "Polygon",
    "coordinates": [
        [
            [0.08905898091569497, 52.69722175598818],
            [0.08905898091569497, 52.15527412683906],
            [0.9565339502005088, 52.15527412683906],
            [0.9565339502005088, 52.69722175598818],
            [0.08905898091569497, 52.69722175598818],
        ]
    ],
}

j = ades.get_process("s1-coherence-d").execute(
    {
        "username": os.getenv("SH_USERNAME"),
        "password": os.getenv("SH_PASSWORD"),
        "intersects": json.dumps(intersects),
        "start_datetime": "2023-04-01",
        "end_datetime": "2023-06-30",
    }
)